# import des aires de services de BD CARTO

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
NATURE = ['Carrefour',
 'Aire de repos ou de service',
 'Péage',
 'Arrêt voyageurs',
 'Port',
 'Aérogare',
 'Gare voyageurs uniquement',
 'Gare voyageurs et fret',
 'Gare fret uniquement',
 'Gare maritime']
NAT_DETAIL = ['Echangeur partiel',
 'Echangeur',
 'Aire de service',
 'Echangeur complet',
 None,
 'Arrêt touristique saisonnier',
 'Port de plaisance',
 'Aire de repos',
 'Gare TGV']
ECHANGEURS = ['Echangeur partiel', 'Echangeur', 'Echangeur complet']
# AIRES = ['Aire de service', 'Aire de repos']
AIRES = ['Aire de service']
BDCARTO = "../../BDCARTO/"
FILE_DATA_CARTO = "/TRANSPORT/EQUIPEMENT_DE_TRANSPORT.shx"
QUALICHARGE_RTET = "../data/BDCARTO/"

In [3]:
# nature = list(pd.unique(noeuds['NATURE']))
# nat_det =list(pd.unique(noeuds['NAT_DETAIL']))

In [4]:
def genere_nature_json(all_noeuds, region, type_noeud, save=True):

    noeuds_nat = all_noeuds.set_index('NAT_DETAIL')
    noeuds_gpd = None
    
    if type_noeud == 'aires' and set(AIRES) & set(list(pd.unique(all_noeuds['NAT_DETAIL']))):  
        noeuds = noeuds_nat.loc[AIRES, ['ID', 'geometry']]  # -> 75
        noeuds_gpd = gpd.GeoDataFrame(noeuds, crs=2154).reset_index()
        
    elif type_noeud == 'echangeurs' and set(ECHANGEURS) & set(list(pd.unique(all_noeuds['NAT_DETAIL']))):
        noeuds = noeuds_nat.loc[ECHANGEURS, ['ID', 'geometry']]  # -> 75
        noeuds_gpd = gpd.GeoDataFrame(noeuds, crs=2154).reset_index()

    if save and noeuds_gpd is not None:
        noeuds_gpd.to_file(QUALICHARGE_RTET + "noeuds_" + type_noeud + "_" + region + ".geojson", driver= "GeoJSON")
    
    return noeuds_gpd

In [7]:
regions = ["aquitaine", "bourgogne", "bretagne", "centre", "corse", "grand_est", "hauts_de_france", 
           "ile_de_france", "loire", "normandie", "occitanie", "PACA", "rhone_alpes"]
type_noeud = "aires"
first = True

for region in regions:
    noeuds = gpd.read_file(BDCARTO + region + FILE_DATA_CARTO)
    #print(noeuds)
    noeuds_gpd = genere_nature_json(noeuds, region, type_noeud)
    if noeuds_gpd is None:
        continue
    if first:
        all_regions = noeuds_gpd
    else:
        all_regions = pd.concat([all_regions, noeuds_gpd])
    first=False
    print(region, ' : ', len(noeuds_gpd), len(all_regions))

all_regions.to_file(QUALICHARGE_RTET + "noeuds_" + type_noeud + "_all.geojson", driver= "GeoJSON")

aquitaine  :  66 66
bourgogne  :  40 106
bretagne  :  21 127
centre  :  36 163
grand_est  :  51 214
hauts_de_france  :  40 254
ile_de_france  :  42 296
loire  :  32 328
normandie  :  18 346
occitanie  :  42 388
PACA  :  37 425
rhone_alpes  :  84 509
